# Topic
## Introduction


This notebook is about the data-serialization language YAML and the template language jinja2.

These two topics are also not covered in the book but we believe you should definitely add these two topics to your python basic knowledge portfolio. You can find much information in the internet and we recommend starting at the official webpages: 
* [YAML](https://yaml.org/)
* [Jinja](https://jinja.palletsprojects.com/)
* [Real Python: Tutorial on Jinja](https://realpython.com/primer-on-jinja-templating/)


## Summary

### YAML (YAML Ain't Markup Language)

Like JSON, XML and many others, YAML is a data format to store and exchange data. As you already know JSON, we will use it to compare it with YAML. Like Python, YAML uses indentation so always keep an eye on your number of spaces ;).

The following describes a sequence of scalars. In YAML-terms, a sequence is a list and scalars are strings or numbers.
```yaml
- Learning Python
- Learning YAML
- Learning Jinja
```

YAML is not in the Python standard library so we need to install it before we can use it. In this class we will use the package [PyYAML](https://pypi.org/project/PyYAML/) and we need to install it with `pip`:

In [ ]:
!pip install pyyaml

In the code below we load the yaml data into python and now we can work with it like a normal python data structure. Instead of `my_yaml`, the data can also be in a file that contains the list.

In [2]:
import yaml
import json

my_yaml = """
- Learning Python
- Learning YAML
- Learning Jinja
"""

data = yaml.safe_load(my_yaml)
print(data)

print(f"{8 * '='} convert to json {8 * '='}")
print(json.dumps(data, indent=4))

['Learning Python', 'Learning YAML', 'Learning Jinja']
======== convert to json ========
[
    "Learning Python",
    "Learning YAML",
    "Learning Jinja"
]


In a next step, we dump it as JSON:

In [3]:
def yaml2json(input_data, verbose=False):
    if verbose:
        print(f"yaml2json received data:\n{input_data}")
    data = yaml.safe_load(input_data)
    if verbose:
        print(f"yaml2json yaml load returned: {data}")
    return json.dumps(data, indent=4)

In [4]:
my_yaml = """
- 1
- 2
- "3"
"""
print(yaml2json(my_yaml))

[
    1,
    2,
    "3"
]


In [5]:
print(yaml2json(my_yaml, verbose=True))

yaml2json received data:

- 1
- 2
- "3"

yaml2json yaml load returned: [1, 2, '3']
[
    1,
    2,
    "3"
]


Now we have written a function to convert YAML to JSON and used another sequence of scalars to test it. The first two numbers are integers and the last one is a string because of the `"`. Also we used the `safe_load` function instead of `load` function. How can we verify the data types and see the difference between the two functions?

In [6]:
test_type = yaml.safe_load(my_yaml)
for scalar in test_type:
    print(type(scalar))

<class 'int'>
<class 'int'>
<class 'str'>


In [7]:
help(yaml.load)

Help on function load in module yaml:

load(stream, Loader=None)
    Parse the first YAML document in a stream
    and produce the corresponding Python object.



In [8]:
help(yaml.safe_load)

Help on function safe_load in module yaml:

safe_load(stream)
    Parse the first YAML document in a stream
    and produce the corresponding Python object.
    
    Resolve only basic YAML tags. This is known
    to be safe for untrusted input.



And now what is the difference between `load` and `safe_load`? With `safe_load` only native data types are supported. **The `load` function can be easily exploited. Please always use `safe_load`**.

A mapping in yaml is equivalent to a dictionary in Python, and we cam map scalars to scalars like this:

In [9]:
my_yaml = """
name: INS
city: Rapperswil
"""
print(yaml2json(my_yaml))

{
    "name": "INS",
    "city": "Rapperswil"
}


Scalars can also be mapped to sequences:

In [10]:
my_yaml = """
trunk:
  - Gig 1/0/1
  - Gig 1/0/2
  - Gig 2/0/1
blocking:
  - Gig 3/0/1
  - Gig 3/0/2
"""
print(yaml2json(my_yaml))

{
    "trunk": [
        "Gig 1/0/1",
        "Gig 1/0/2",
        "Gig 2/0/1"
    ],
    "blocking": [
        "Gig 3/0/1",
        "Gig 3/0/2"
    ]
}


Sequences of mappings are also possible:

In [11]:
my_yaml = """
- name: Gig 1/0/1
  mode: L2
  enabled: True
  switchport: access
  vlan: 10
- name: Gig 1/0/2
  mode: L3
  enabled: True
  ip: 192.168.1.1/24
- name: Gig 1/0/3
  mode: L2
  enabled: False 
  switchport: trunk
"""
print(yaml2json(my_yaml))

[
    {
        "name": "Gig 1/0/1",
        "mode": "L2",
        "enabled": true,
        "switchport": "access",
        "vlan": 10
    },
    {
        "name": "Gig 1/0/2",
        "mode": "L3",
        "enabled": true,
        "ip": "192.168.1.1/24"
    },
    {
        "name": "Gig 1/0/3",
        "mode": "L2",
        "enabled": false,
        "switchport": "trunk"
    }
]


Now let's get creative and have mapping of mappings:

In [12]:
my_yaml = """
Gig1/0/1:
  enabled: True
  description: uplink

Gig1/0/2:
  enabled: True
  description: Printer pr001

"""
print(yaml2json(my_yaml))

{
    "Gig1/0/1": {
        "enabled": true,
        "description": "uplink"
    },
    "Gig1/0/2": {
        "enabled": true,
        "description": "Printer pr001"
    }
}


... and sequences of sequences!

In [13]:
my_yaml = """
-
  - permit
  - ip
  - 10.1.1.0 0.0.0.255
  - 172.16.1.0 0.0.0.255
-
  - permit
  - ip
  - 10.1.2.0 0.0.0.255
  - 172.16.1.0 0.0.0.255
  - eq www
-
  - deny
  - ip
  - any
  - any
"""
print(yaml2json(my_yaml))

[
    [
        "permit",
        "ip",
        "10.1.1.0 0.0.0.255",
        "172.16.1.0 0.0.0.255"
    ],
    [
        "permit",
        "ip",
        "10.1.2.0 0.0.0.255",
        "172.16.1.0 0.0.0.255",
        "eq www"
    ],
    [
        "deny",
        "ip",
        "any",
        "any"
    ]
]


This is just a short introduction and covers only the tip of the iceberg. For more detailed information please check out [the YAML website](https://yaml.org/).

### Jinja2

With the string manipulation methods you already know it is easy to render some text. Especially the `f-String` or `format` function are handy, but when it comes to longer texts we need a better solution. Jinja2 is a template engine for python. It makes it easy to render all kind of texts like web pages, reports or configuration files. First we need to install the library via `pip`:

In [ ]:
!pip install jinja2

In [15]:
import jinja2

template = jinja2.Template("My name is {{ name }} and I like {{ pizza }} pizza")
template.render(name="Max", pizza="Margherita ")

'My name is Max and I like Margherita  pizza'

This basic example could be easily done with an `f-string` but jinja2 can do much more. The delimiter `{{ ... }}` is used for an expression to access a variable and print it to the template output. Statements are surrounded by `{% ... %}` and are used for example for special control structure statements. Comments can be written by using `{# ... #}`.

In [16]:
from jinja2 import Template

template_raw = """
Dear {{ name }},

I need the following list from the store:
{% for item in shopping_list %}
  - {{ item }}
{% endfor %}

All the best!
"""
template = Template(template_raw)
groceries = ["Apple", "Pear", "Salad", "Beer", "More beer"]
text = template.render(name="John", shopping_list=groceries)
print(text)


Dear John,

I need the following list from the store:

  - Apple

  - Pear

  - Salad

  - Beer

  - More beer


All the best!


Jinja2 is really powerful and we cover only the basics here. Please take a look at the official [Template Designer Documentation](https://jinja.palletsprojects.com/en/2.11.x/templates/) for more information.

## Exercises

### Exercise 1: JSON to YAML
Write a yaml file `exercise1.yaml` equal to the following JSON.

```JSON
{
  "names": [
    "Emilia",
    "Kit",
    "Maisie",
    "Peter",
    "Sophie",
    "Jason"
  ],
  "numbers": [
    1,
    2,
    1.5,
    2.0
  ],
  "boolean": true
}
```

In [ ]:
%%writefile exercise1.yaml
# Todo: write the yaml file

In [ ]:
import yaml

with open("exercise1.yaml") as f:
    data = yaml.safe_load(f.read())

assert data["names"] == ["Emilia", "Kit", "Maisie", "Peter", "Sophie", "Jason"]
assert data["numbers"] == [1, 2, 1.5, 2.0]
assert isinstance(data["numbers"][3], float)
assert data["boolean"] == True
print("Data are okay")

### Exercise 2: Song-Template
Write a template to match the following output:

```
Old MacDonald had a farm, E I E I O,
And on his farm he had a cow, E I E I O.
With a moo moo here and a moo moo there,
Here a moo, there a moo, everywhere a moo moo.
Old MacDonald had a farm, E I E I O.

Old MacDonald had a farm, E I E I O,
And on his farm he had a pig, E I E I O.
With a oink oink here and a oink oink there,
Here a oink, there a oink, everywhere a oink oink.
Old MacDonald had a farm, E I E I O.

Old MacDonald had a farm, E I E I O,
And on his farm he had a duck, E I E I O.
With a quack quack here and a quack quack there,
Here a quack, there a quack, everywhere a quack quack.
Old MacDonald had a farm, E I E I O.
```

In [ ]:
from jinja2 import Template

template_raw = """
{# Todo: write the template #}
"""

animals = {"cow": "moo", "pig": "oink", "duck": "quack"}
text = Template(template_raw).render(animals=animals)

print(text)

### Exercise 3: Train Connections
Write a python script to render train connections. The script should take the arguments `-s` (`--start`) and `-e` (`--end`). Use the open data api to get the data:

https://transport.opendata.ch/docs.html#connections 

Please install the `requests` library via `pip` to make the API calls to the link above.

```
$ train_connections.py -s Rapperswil -e Herisau

Connections
===========

From: Rapperswil		To: Herisau
2020-07-20T05:33:00+0200	Platform: 5

From: Rapperswil		To: Herisau
2020-07-20T06:03:00+0200	Platform: 2

From: Rapperswil		To: Herisau
2020-07-20T06:33:00+0200	Platform: 5

From: Rapperswil		To: Herisau
2020-07-20T07:03:00+0200	Platform: 2
```

In [ ]:
%%writefile train_connections.py
import requests
# todo: imports

TEMPLATE = """
{# ToDo: write template #}
"""
def main(start, end):
    try:
        response = requests.get(f"https://transport.opendata.ch/v1/connections?from={start}&to={end}")
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Error {e}")
        exit(1)
    
    data = response.json()
    # ToDo: render template and print

if __name__ == "__main__":
    # ToDo: Create an ArgumentParser and start main

In [ ]:
%run train_connections.py -s Rapperswil -e Herisau